# Homework-4
Recurrent Neural Networks  

## Problem 1:

Answer each of the following questions.

(A) Both CNNs and RNNs share parameters—but in different ways. Explain.

INSERT ANSWER HERE

CNN parameters are used for dimensional problems such as images and videos, which kernel applies to input(image,video), allowing weights to find patterns without considering position.
RNN are more used in sequential data(weather forecast,stock),same weights are used at every time step in the sequence.  


(B) What is the basic idea underlying how we learn meaningful word embeddings? How does this relate to the idea of self-supervised learning?

INSERT ANSWER HERE

basic structure of word embedding is predicting word from its context, finding similarity. to do this word embedding place words as vectorized space, and find similarity from the word context. and word embedding does not labeled for prediction(ex, 1 or 0 ) since its struture is not finding yes or no for similarity. and self supervise learning's structure is nto derive various supervision signals based on the data, which alignes with what word embedding does. 


(C) Consider an LSTM layer that accepts a batch of arbitrary size, with 40 time steps and 3 features. The LSTM layer has input shape `[None, 40, 3]`. Suppose its output is shape `[None, 2]`. Find the number of parameters in the LSTM layer, explaining your thinking.

INSERT ANSWER HERE

input shape has 40 sequence(time steps) and each has three features. output shape has size 2, which LSTM has two units, also four gates for LSTM.
weight for input( 3 features) : 3*2 = 6 ; 3 weights per unit and there are two units so 3*2 = 6 units per gate.  hidden state weight : 2*2 ; two weights per unit and there are two units wich leads to four weights per gate. two bias per gate.
so, 4(6+4+2) = 48

total params : 48


## Problem-2: 

 Begin by reviewing Andrej Karpathy's famous blog post [The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/).

Provide a three-paragraph summary of the article, describing Karpathy's main points.

INSERT ANSWER HERE

## Problem-3: 

Write code for a character-based RNN (i.e. LSTM or GRU, not simple_RNN) in PyTorch. You can use his source code and any code online to assist. If you do, just reference where you sought help from.

- Choose a large text corpus, such as a collection of novels from [Project Gutenburg](https://www.gutenberg.org/). You can use other data, but you should explain where your data comes from.

INSERT ANSWER HERE

- Perform any necessary preprocessing, explaining what steps you take. In particular, what forms of normalization do you use? Do you define characters with special meaning?

for preprocessing, it has processed characters to be lowercase(not case sensitive), removing witespace.

In [ ]:
# INSERT CODE
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import random
import string

with open("mobi.rtf", encoding='utf-8') as f:
    raw_text = f.read()

text = raw_text.lower()
all_chars = sorted(list(set(c for c in text if c in string.printable)))
vocab_size = len(all_chars)

char_idx = {ch: idx for idx, ch in enumerate(all_chars)}
idx_char = {idx: ch for ch, idx in char_idx.items()}

def encode(text):
    return [char_idx[c] for c in text if c in char_idx]

def decode(indices):
    return ''.join(idx_char[i] for i in indices)

encoded_text = encode(text)

- Efficiently load and batch the dataset for training using a `DataLoader`. Make sure to reserve some of the data for validation and testing. Describe how you handle batching and sequence lengths.

In [ ]:
# INSERT CODE
import torch
from torch.utils.data import Dataset, DataLoader, random_split

encoded_text = encode(text)
encoded_text = torch.tensor(encoded_text, dtype=torch.long)

n = len(encoded_text)
train_len = int(0.8 * n)
val_len = int(0.05 * n)
test_len = n - train_len - val_len

train_seq = encoded_text[:train_len]
val_seq = encoded_text[train_len:train_len+val_len]
test_seq = encoded_text[train_len+val_len:]


class CharDataset(Dataset):
    def __init__(self, data, seq_len):
        self.data = data
        self.seq_len = seq_len

    def __len__(self):
        return len(self.data) - self.seq_len

    def __getitem__(self, idx):
        x = self.data[idx:idx+self.seq_len]
        y = self.data[idx+1:idx+self.seq_len+1]
        return x, y

SEQ_LEN = 100
BATCH_SIZE = 64

train_dataset = CharDataset(train_seq, SEQ_LEN)
val_dataset = CharDataset(val_seq, SEQ_LEN)
test_dataset = CharDataset(test_seq, SEQ_LEN)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)



- Define your RNN model. Discuss the number of layers, hidden units, and the type of RNN cells you use. What is the total number of parameters in your model? Explain the rationale behind your architectural choices.

In [ ]:
# INSERT CODE
import torch.nn as nn

class RNN1(nn.Module):
    def __init__(self, vocab_size, embedding_dim=256, hidden_size=512, num_layers=2, rnn_type='lstm'):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)

        if rnn_type == 'lstm':
            self.rnn = nn.LSTM(input_size=embedding_dim,
                               hidden_size=hidden_size,
                               num_layers=num_layers,
                               batch_first=True)
        elif rnn_type == 'gru':
            self.rnn = nn.GRU(input_size=embedding_dim,
                              hidden_size=hidden_size,
                              num_layers=num_layers,
                              batch_first=True)
        
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, x, hidden=None):
        x = self.embedding(x)                  
        output, hidden = self.rnn(x, hidden)    
        output = self.fc(output)              
        return output, hidden




- Write the training loop.

In [ ]:
# INSERT CODE
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm

# --- Hyperparameters ---
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
EPOCHS = 3
CLIP_GRAD = 5.0  # to prevent exploding gradients

# --- Initialize model, loss, optimizer ---
vocab_size = len(char2idx)
model = RNN1(vocab_size=vocab_size, embedding_dim=256, hidden_size=512, num_layers=2, rnn_type='lstm').to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.003)

# --- Training ---
for epoch in range(1, EPOCHS + 1):
    model.train()
    total_loss = 0
    pbar = tqdm(train_loader, desc=f"Epoch {epoch}")

    for inputs, targets in pbar:
        inputs, targets = inputs.to(device), targets.to(device)

        # Forward pass
        outputs, _ = model(inputs)
        loss = criterion(outputs.view(-1, vocab_size), targets.view(-1))

        # Backward + Optimize
        optimizer.zero_grad()
        loss.backward()
        if CLIP_GRAD:
            nn.utils.clip_grad_norm_(model.parameters(), CLIP_GRAD)
        optimizer.step()

        total_loss += loss.item()
        pbar.set_postfix(loss=loss.item())

    avg_train_loss = total_loss / len(train_loader)

    # --- Validation ---
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for inputs, targets in val_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs, _ = model(inputs)
            loss = criterion(outputs.view(-1, vocab_size), targets.view(-1))
            val_loss += loss.item()
    avg_val_loss = val_loss / len(val_loader)

    print(f"Epoch {epoch} | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")



Epoch 1: 100%|██████████| 179/179 [00:58<00:00,  3.07it/s, loss=0.119]


Epoch 1 | Train Loss: 1.1010 | Val Loss: 3.3128


Epoch 2: 100%|██████████| 179/179 [00:55<00:00,  3.22it/s, loss=0.0995]


Epoch 2 | Train Loss: 0.1039 | Val Loss: 3.6960


Epoch 3: 100%|██████████| 179/179 [00:54<00:00,  3.25it/s, loss=0.0929]


Epoch 3 | Train Loss: 0.0913 | Val Loss: 3.8918



- Monitor and report on the training progress by tracking the loss. After training, evaluate the model's performance using a suitable evaluation metric (e.g., perplexity) on a validation dataset or a held-out portion of the training data. Discuss the results.

In [9]:
# INSERT CODE
import math
import matplotlib.pyplot as plt

train_losses = []
val_losses = []
val_perplexities = []

for epoch in range(1, EPOCHS + 1):
    model.train()
    total_loss = 0
    for inputs, targets in tqdm(train_loader, desc=f"Epoch {epoch}"):
        inputs, targets = inputs.to(device), targets.to(device)
        outputs, _ = model(inputs)
        loss = criterion(outputs.view(-1, vocab_size), targets.view(-1))

        optimizer.zero_grad()
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), CLIP_GRAD)
        optimizer.step()

        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_loader)
    train_losses.append(avg_train_loss)

    # --- Validation ---
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for inputs, targets in val_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs, _ = model(inputs)
            loss = criterion(outputs.view(-1, vocab_size), targets.view(-1))
            val_loss += loss.item()

    avg_val_loss = val_loss / len(val_loader)
    val_losses.append(avg_val_loss)
    perplexity = math.exp(avg_val_loss)
    val_perplexities.append(perplexity)

    print(f"Epoch {epoch} | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f} | Perplexity: {perplexity:.2f}")



Epoch 1: 100%|██████████| 179/179 [00:57<00:00,  3.12it/s]


Epoch 1 | Train Loss: 0.0867 | Val Loss: 4.3067 | Perplexity: 74.19


Epoch 2: 100%|██████████| 179/179 [00:55<00:00,  3.25it/s]


Epoch 2 | Train Loss: 0.0836 | Val Loss: 4.1532 | Perplexity: 63.64


Epoch 3: 100%|██████████| 179/179 [00:57<00:00,  3.11it/s]


Epoch 3 | Train Loss: 0.0817 | Val Loss: 4.2255 | Perplexity: 68.41



- Specify the hyper-parameters (for example, model hyper-parameters, as well as sampling size and beam width from below) used in your model. Find suitable settings using a validation split.

In [ ]:
# INSERT CODE
hyperparams = {
    'embedding_dim': 256,
    'hidden_size': 512,
    'num_layers': 2,
    'rnn_type': 'lstm',
    'seq_len': 100,
    'batch_size': 64,
    'learning_rate': 0.003,
    'epochs': 20,
    'temperature': 0.8,
    'sampling_length': 500,
    'clip_grad_norm': 5.0
}



- Implement a text generation function using the trained RNN model. Provide a prompt or seed text, and use the RNN to generate a sequence of characters. Experiment with different prompt texts and observe how the generated text changes. Discuss any interesting patterns or observations you make during the text generation process.

In [ ]:
# INSERT CODE
import torch.nn.functional as F

def generate_text(model, start_text='start', length=500, temperature=0.8, device='cpu'):
    model.eval()
    input_indices = [char_idx[c] for c in start_text.lower() if c in char_idx]
    input_tensor = torch.tensor([input_indices], dtype=torch.long).to(device)

    generated = start_text
    hidden = None

    with torch.no_grad():
        for _ in range(length):
            output, hidden = model(input_tensor, hidden)
            logits = output[:, -1, :] / temperature  # shape: (1, vocab_size)
            probs = F.softmax(logits, dim=-1).squeeze()
            next_char_idx = torch.multinomial(probs, num_samples=1).item()
            next_char = idx_char[next_char_idx]
            generated += next_char
            input_tensor = torch.tensor([[next_char_idx]], dtype=torch.long).to(device)

    return generated

print(generate_text(model, start_text="Call me Ishmael. ", temperature=0.8, length=500))
print()
print(generate_text(model, start_text="The ocean", temperature=0.7, length=500))
print()
print(generate_text(model, start_text="Whale", temperature=1.0, length=500))


Call me Ishmael. some years ago\'97never mind how long precisely\'97having\
little or no money in my purse, and nothing particular to interest me\
on shore, i thought i would sail about a little and see the watery part\
of the world. it is a way i have of driving off the spleen and\
regulating the circulation. whenever i find myself growing grim about\
the mouth; whenever it is a damp, drizzly november in my soul; whenever\
i find myself involuntarily pausing before coffin warehouses, and\
bringing up the rear o

The ocean with me.\
\
there now is your insular city of the manhattoes, belted round by\
wharves as indian isles by coral reefs\'97commerce surrounds it with her\
surf. right and left, the streets take you waterward. its extreme\
downtown is the battery, where that noble mole is washed by waves, and\
cooled by breezes, which a few hours previous were out of sight of\
land. look at the crowds of water-gazers there.\
\
circumambulate the city of a dreamy sabbath afternoon. go fr

## Problem-4: Optional extra credit 
 
Up to +3 bonus points 

* Repeat the text generation process from the previous problem, but do it with a transformer architecture rather than an LSTM/GRU (you can use word tokens instead of character tokens if you prefer). Use the provided mini-GPT lab for reference.  

## Problem-5: Optional extra credit 
 


- (+1 bonus points): Now with your model trained, implement top-k and nucleus sampling. Again, provide a prompt or seed text, and use the RNN to generate a sequence of characters. Experiment with different prompt texts and observe how the generated text changes. Discuss any interesting patterns or observations you make during the text generation process.

In [ ]:
# INSERT CODE

- (+1 bonus points): Now with your model trained, implement beam search. Again, provide a prompt or seed text, and use the RNN to generate a sequence of characters. Experiment with different prompt texts and observe how the generated text changes. Discuss any interesting patterns or observations you make during the text generation process.  

In [ ]:
# INSERT CODE